In [1]:
from preprocess import load_data, preprocess_hepatitis_data

In [2]:
train_dfs = load_data('datasetsCBR/hepatitis/hepatitis.fold.*.train.arff')
test_dfs = load_data('datasetsCBR/hepatitis/hepatitis.fold.*.test.arff')

train_dfs[0].head()


,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER_BIG,LIVER_FIRM,SPLEEN_PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK_PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY,Class
0,50.0,b'female',b'no',b'no',b'yes',b'no',b'no',b'no',b'no',b'no',b'no',b'no',b'no',0.9,135.0,42.0,3.5,NaN,b'no',b'LIVE'
1,45.0,b'male',b'no',b'no',b'yes',b'yes',b'no',b'yes',b'no',b'yes',b'no',b'no',b'no',1.0,85.0,75.0,NaN,NaN,b'no',b'LIVE'
2,54.0,b'female',b'no',b'no',b'yes',b'yes',b'no',b'?',b'?',b'yes',b'no',b'yes',b'no',3.9,120.0,28.0,3.5,43.0,b'yes',b'DIE'
3,35.0,b'female',b'no',b'no',b'yes',b'no',b'no',b'?',b'?',b'yes',b'yes',b'yes',b'no',1.5,138.0,58.0,2.6,NaN,b'yes',b'DIE'
4,24.0,b'female',b'no',b'no',b'yes',b'no',b'no',b'yes',b'no',b'no',b'no',b'no',b'no',1.0,NaN,34.0,4.1,NaN,b'yes',b'LIVE'
